In [1]:
#pip install -q opencv-python-headless

# Signature Extraction Function

In [9]:
import numpy as np
import cv2
#from google.colab.patches import cv2_imshow

def extract_signature(filename:str):
  # Lee la imagen y la guarda en la variable image
  image = cv2.imread(filename)
  # Vamos a copiar la imagen original en la variable `original` 
  original = image.copy()
  # Vamos a concentrarnos en la parte inferior izquierda de este cheque cortando la imagen
  h, w, c = image.shape
  print('ancho:  ', w)
  print('alto: ', h)
  print('canales:', c)
  # Debido a que el cheque mide 600 de ancho, 264 de alto, vamos a dividir el cheque en cuatro partes iguales
  # y nos quedaremos solo con la esquina inferior derecha.
  image = image[ 131:, 299:]
  result = image.copy()
  # Convertiremos la imagen de BGR a HSV (también conocido como HSL). 
  image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  # Aquí haremos un filtro basado en un rango inferior y superior dados los siguientes valores de HSV. 
  lower = np.array([0, 0, 0])
  upper = np.array([255, 255, 80])
  mask = cv2.inRange(image, lower, upper)

  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
  # En una imagen binaria la erosión hace que un pixel siga siendo 1, si y solo si los pixeles
  # de alrededor son también 1
  erosion = cv2.erode(mask,kernel,iterations = 1)
  # Hace lo opuesto de la erosión. Aquí un pixel es 1 si al menos un pixel debajo del kernel es 1
  dilation = cv2.dilate(erosion,kernel,iterations = 1)
  # Otras opciones que se pueden aplicar a la imagen
  #opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
  #close = cv2.morphologyEx(erosion, cv2.MORPH_CLOSE, kernel, iterations=1)

  print("Original image","\n")
  cv2.imshow(original)
  print("dilation","\n")
  cv2.imshow( dilation)
  print("Next cheque","\n","\n","\n","\n")

  output=filename.split("/")[-1].replace(".jpeg","")
  cv2.imwrite(f'solo_dilation/dilation_{output}.png', dilation)

In [10]:
import glob

for filepath in glob.iglob('/content/cheques/*.jpeg'):
    extract_signature(filepath)

In [11]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

def extract_signature_longer(filename:str):
  # Lee la imagen y la guarda en la variable image
  image = cv2.imread(filename)
  # Vamos a copiar la imagen original en la variable `original` 
  original = image.copy()
  # Vamos a concentrarnos en la parte inferior izquierda de este cheque cortando la imagen
  h, w, c = image.shape
  print('ancho:  ', w)
  print('alto: ', h)
  print('canales:', c)
  # Debido a que el cheque mide 600 de ancho, 264 de alto, vamos a dividir el cheque en cuatro partes iguales
  # y nos quedaremos solo con la esquina inferior derecha.
  image = image[ 131:, 299:]
  result = image.copy()
  # Convertiremos la imagen de BGR a HSV (también conocido como HSL). 
  image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
  # Aquí haremos un filtro basado en un rango inferior y superior dados los siguientes valores de HSV. 
  lower = np.array([0, 0, 0])
  upper = np.array([255, 255, 80])
  mask = cv2.inRange(image, lower, upper)

  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
  # En una imagen binaria la erosión hace que un pixel siga siendo 1, si y solo si los pixeles
  # de alrededor son también 1
  erosion = cv2.erode(mask,kernel,iterations = 1)
  # Hace lo opuesto de la erosión. Aquí un pixel es 1 si al menos un pixel debajo del kernel es 1
  dilation = cv2.dilate(erosion,kernel,iterations = 1)
  # Otras opciones que se pueden aplicar a la imagen
  #opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
  #close = cv2.morphologyEx(erosion, cv2.MORPH_CLOSE, kernel, iterations=1)


  # Lo siguiente sirve para crear rectángulos alrededor de las firmas. 
  # no será utilizado en realidad, pero me pareció elegante y útil
  cnts = cv2.findContours(dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  boxes = []
  for c in cnts:
      (x, y, w, h) = cv2.boundingRect(c)
      boxes.append([x,y, x+w,y+h])
  boxes = np.asarray(boxes)
  left = np.min(boxes[:,0])
  top = np.min(boxes[:,1])
  right = np.max(boxes[:,2])
  bottom = np.max(boxes[:,3])
  result[dilation==0] = (255,255,255)
  ROI = result[top:bottom, left:right].copy()
  cv2.rectangle(result, (left,top), (right,bottom), (36, 255, 12), 2)

  print("Original image","\n")
  cv2_imshow(original)
  #print("Result","\n")
  #cv2_imshow( result)
  #print("Region of interest","\n")
  #cv2_imshow( ROI)
  print("dilation","\n")
  cv2_imshow( dilation)
  print("Next cheque","\n","\n","\n","\n")

  output=filename.split("/")[-1].replace(".jpeg","")
  cv2.imwrite(f'solo_dilation/dilation_{output}.png', dilation)
  cv2.imwrite(f'solo_ROI/ROI_{output}.png', ROI)
  cv2.waitKey()

ModuleNotFoundError: No module named 'google.colab'